In [ ]:
# Install required packages
!pip install ultralytics opencv-python numpy scikit-learn filterpy




In [ ]:
# Mount Google Drive (if needed)
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from ultralytics import YOLO
import torch
model = YOLO("yolov8m.pt")  # Load YOLOv8 Medium Model

model.train(
    data="/content/drive/MyDrive/yolo dataset/data.yaml",
    epochs=25,         # First training phase (25 epochs)
    imgsz=640,         # Image size
    batch=16,          # Batch size
    conf=0.5,          # Confidence threshold
    iou=0.45,          # IoU threshold for NMS
    optimizer="SGD",   # Stochastic Gradient Descent optimizer
    lr0=0.01,          # Initial learning rate
    augment=True       # Enable data augmentation
)


In [ ]:
from ultralytics import YOLO
# Load last trained model from Google Drive
model = YOLO("/content/drive/MyDrive/last.pt")  # Update with the correct path

# Continue training for another 25 epochs
model.train(
    data="/content/drive/MyDrive/yolo dataset/data.yaml",
    epochs=25,         # Continue from epoch 26 to 50
    imgsz=640,
    batch=16,
    conf=0.5,
    iou=0.45,
    optimizer="SGD",
    lr0=0.01,
    augment=True
)


In [ ]:
from ultralytics import YOLO

# Load trained YOLOv8 model
model = YOLO("/content/drive/MyDrive/best (2).pt")

# Run validation
metrics = model.val(data="/content/drive/MyDrive/yolo dataset/data.yaml", iou=0.5)

# Extract and print correct metrics
print(f"🔹 mAP@0.5: {metrics.box.map:.2%}")  # Mean Average Precision at IoU 0.5
print(f"🔹 Precision: {metrics.box.mp:.2%}")  # Mean Precision
print(f"🔹 Recall: {metrics.box.mr:.2%}")  # Mean Recall

# Correct FPS calculation
fps = 1000 / metrics.speed["inference"]  # Convert ms per image to FPS
print(f"🔹 FPS: {fps:.2f}")  # Frames per second


In [ ]:
import numpy as np
from filterpy.kalman import KalmanFilter

# Create standard Kalman Filter
def create_kf():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1, 1, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 1],
                     [0, 0, 0, 1]])
    kf.H = np.array([[1, 0, 0, 0],
                     [0, 0, 1, 0]])
    kf.P *= 1000
    kf.R *= 5
    return kf

# Create Extended Kalman Filter (EKF)
def create_ekf():
    ekf = create_kf()
    ekf.Q *= 0.1
    return ekf

# Create Unscented Kalman Filter (UKF)
def create_ukf():
    ukf = create_kf()
    ukf.Q *= 0.5
    return ukf


In [ ]:
from sklearn.svm import SVC
import joblib

# Generate synthetic motion data (velocity, acceleration, direction change)
X_train = np.array([
    [10, 0, 0],   # KF (Uniform motion)
    [20, 5, 0],   # EKF (Accelerated motion)
    [30, 10, 20], # UKF (Nonlinear motion)
    [15, 2, 5],   # SVM Adaptive
    [25, 6, 15],  # SVM Adaptive
])

y_train = np.array([0, 1, 2, 3, 3])  # 0=KF, 1=EKF, 2=UKF, 3=SVM Adaptive

# Train SVM model
svm_model = SVC(kernel="rbf", C=1.0, gamma="scale")
svm_model.fit(X_train, y_train)

# Save the trained model
joblib.dump(svm_model, "svm_filter_selector.pkl")

print("✅ SVM-based filter selection model trained and saved!")


In [ ]:
import joblib

# Load trained SVM model
svm_model = joblib.load("svm_filter_selector.pkl")

def select_filter(velocity, acceleration, direction_change):
    features = np.array([[velocity, acceleration, direction_change]])
    label = svm_model.predict(features)[0]

    if label == 0:
        return create_kf()  # Standard Kalman Filter
    elif label == 1:
        return create_ekf()  # Extended Kalman Filter
    elif label == 2:
        return create_ukf()  # Unscented Kalman Filter
    else:
        return create_kf()  # Default to KF

print("✅ Adaptive filter selection function ready!")


In [ ]:
def error_feedback(kf, tracking_error):
    if tracking_error > 10:  # High error
        kf.Q *= 1.2  # Increase process noise
        kf.R *= 1.1  # Increase measurement noise
    elif tracking_error < 2:  # Low error
        kf.Q *= 0.8  # Decrease process noise
        kf.R *= 0.9  # Decrease measurement noise
    return kf


In [ ]:
!pip install --upgrade pip setuptools wheel  # Upgrade pip tools
!pip install scikit-image==0.19.3  # Install a compatible version
!pip install filterpy  # Install filterpy manually


In [ ]:
!git clone https://github.com/abewley/sort.git



In [ ]:
!rm -rf ~/.config/matplotlib
!pip uninstall -y matplotlib
!pip install matplotlib


In [ ]:
import matplotlib
matplotlib.use('Agg')  # Set a headless backend
import matplotlib.pyplot as plt


In [ ]:
import cv2
import numpy as np
import joblib
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter

# Load YOLOv8 model
model = YOLO("/content/drive/MyDrive/best (2).pt")

# Load trained SVM model for filter selection
svm_model = joblib.load("/content/svm_filter_selector.pkl")

# Initialize video capture
video_path = "/content/drive/MyDrive/traffic.mp4"
cap = cv2.VideoCapture(video_path)

# Define video output
output_video_path = "/content/drive/MyDrive/adaptive_tracking.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Initialize tracking dictionary
trackers = {}

# 🔹 Function to Initialize Kalman Filter
def create_kf():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1, 1, 0, 0],  # State transition matrix
                     [0, 1, 0, 0],
                     [0, 0, 1, 1],
                     [0, 0, 0, 1]])

    kf.H = np.array([[1, 0, 0, 0],  # Measurement function
                     [0, 0, 1, 0]])

    kf.P *= 1000  # High uncertainty
    kf.R *= 10    # Measurement noise
    kf.Q *= 0.1   # Process noise
    return kf

# 🔹 Function to Select the Best Kalman Filter Based on Motion Features
def select_filter(velocity, acceleration, direction_change):
    features = np.array([[velocity, acceleration, direction_change]])
    filter_type = svm_model.predict(features)[0]

    if filter_type == 0:
        return create_kf()  # Kalman Filter (KF)
    elif filter_type == 1:
        return create_kf()  # Extended Kalman Filter (Placeholder)
    else:
        return create_kf()  # Unscented Kalman Filter (Placeholder)

# 🔹 Function to Apply Error Feedback Mechanism
def error_feedback(kf, error):
    if error > 20:
        kf.Q *= 1.5  # Increase process noise
    elif error < 5:
        kf.Q *= 0.8  # Decrease process noise
    return kf

# 🔹 Process Video Frame by Frame
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Run YOLOv8 detection
    results = model(frame)

    detections = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            cls = box.cls[0].cpu().numpy()

            # Only track vehicles (Car=0, Truck=1, Bus=2, Train=3)
            if cls in [0, 1, 2, 3]:
                detections.append([x1, y1, x2, y2, conf])

    # Convert to numpy array
    detections = np.array(detections)

    # Tracking Logic
    new_trackers = {}

    for detection in detections:
        x1, y1, x2, y2, conf = detection
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2

        # Compute motion features
        velocity = np.random.uniform(0, 10)  # Placeholder velocity computation
        acceleration = np.random.uniform(0, 5)  # Placeholder acceleration
        direction_change = np.random.uniform(0, 30)  # Placeholder

        # Check if the object is already being tracked
        matched_id = None
        for track_id, kf in trackers.items():
            predicted_state = kf.x[:2]  # Predicted x, y
            dist = np.linalg.norm([center_x - predicted_state[0], center_y - predicted_state[1]])
            if dist < 50:  # Threshold for considering a match
                matched_id = track_id
                break

        if matched_id is not None:
            kf = trackers[matched_id]
        else:
            matched_id = frame_count  # Assign a new track ID
            kf = select_filter(velocity, acceleration, direction_change)
            kf.x[:2] = np.array([[center_x], [center_y]])

        # Predict and update Kalman filter
        state = kf.predict()
        if state is None:
          state = kf.x
        kf.update(np.array([center_x, center_y]))

        # Apply error feedback
        tracking_error = np.linalg.norm([center_x - state[0], center_y - state[1]])
        kf = error_feedback(kf, tracking_error)

        new_trackers[matched_id] = kf

        # Draw tracking results
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {matched_id}", (int(x1), int(y1) - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    trackers = new_trackers  # Update tracker dictionary

    # Save frame
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Adaptive Kalman Tracking Complete! Saved video: {output_video_path}")


In [ ]:
import cv2
import numpy as np
import joblib
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter

# Load YOLOv8 model
model = YOLO("/content/drive/MyDrive/best (2).pt")

# Load trained SVM model for selecting the best Kalman filter
svm_model = joblib.load("/content/svm_filter_selector.pkl")

# Load video
video_path = "/content/drive/MyDrive/traffic.mp4"
cap = cv2.VideoCapture(video_path)

# Define video output
output_video_path = "/content/drive/MyDrive/new_tracking.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Initialize tracking dictionary
trackers = {}

# 🔹 Function to Initialize Kalman Filter
def create_kf():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1, 1, 0, 0],  # State transition matrix
                     [0, 1, 0, 0],
                     [0, 0, 1, 1],
                     [0, 0, 0, 1]])

    kf.H = np.array([[1, 0, 0, 0],  # Measurement function
                     [0, 0, 1, 0]])

    kf.P *= 1000  # High uncertainty
    kf.R *= 10    # Measurement noise
    kf.Q *= 0.1   # Process noise
    return kf

# 🔹 Function to Select the Best Kalman Filter Based on Motion Features
def select_filter(velocity, acceleration, direction_change):
    features = np.array([[velocity, acceleration, direction_change]])
    filter_type = svm_model.predict(features)[0]

    if filter_type == 0:
        return create_kf()  # Kalman Filter (KF)
    elif filter_type == 1:
        return create_kf()  # Extended Kalman Filter (Placeholder)
    else:
        return create_kf()  # Unscented Kalman Filter (Placeholder)

# 🔹 Function to Apply Error Feedback Mechanism
def error_feedback(kf, error):
    if error > 20:
        kf.Q *= 1.5  # Increase process noise
    elif error < 5:
        kf.Q *= 0.8  # Decrease process noise
    return kf

# 🔹 Process Video Frame by Frame
frame_count = 0
next_track_id = 1  # Unique ID for new vehicles

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Run YOLOv8 detection
    results = model(frame)

    detections = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            cls = box.cls[0].cpu().numpy()

            # Only track vehicles (Car=0, Truck=1, Bus=2, Train=3)
            if cls in [0, 1, 2, 3]:
                detections.append([x1, y1, x2, y2, conf])

    # Convert to numpy array
    detections = np.array(detections)

    # Tracking Logic
    new_trackers = {}

    for detection in detections:
        x1, y1, x2, y2, conf = detection
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2

        # Compute motion features
        velocity = np.random.uniform(0, 10)  # Placeholder velocity computation
        acceleration = np.random.uniform(0, 5)  # Placeholder acceleration
        direction_change = np.random.uniform(0, 30)  # Placeholder

        # Check if the object is already being tracked
        matched_id = None
        for track_id, kf in trackers.items():
            predicted_state = kf.x[:2]  # Predicted x, y
            dist = np.linalg.norm([center_x - predicted_state[0], center_y - predicted_state[1]])
            if dist < 50:  # Threshold for considering a match
                matched_id = track_id
                break

        if matched_id is not None:
            kf = trackers[matched_id]
        else:
            matched_id = next_track_id  # Assign a new track ID
            next_track_id += 1
            kf = select_filter(velocity, acceleration, direction_change)
            kf.x[:2] = np.array([[center_x], [center_y]])

        # Predict and update Kalman filter
        state = kf.predict()
        if state is None:
          state = kf.x
        kf.update(np.array([center_x, center_y]))

        # Apply error feedback
        tracking_error = np.linalg.norm([center_x - state[0], center_y - state[1]])
        kf = error_feedback(kf, tracking_error)

        new_trackers[matched_id] = kf

        # Draw tracking results
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, f"ID {matched_id}", (int(x1), int(y1) - 5),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    trackers = new_trackers  # Update tracker dictionary

    # Save frame
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Adaptive Kalman Tracking Complete! Saved video: {output_video_path}")


In [ ]:
import cv2
import numpy as np
import joblib
from ultralytics import YOLO
from filterpy.kalman import KalmanFilter

# Load YOLOv8 model
model = YOLO("/content/drive/MyDrive/best (2).pt")

# Load trained SVM model for filter selection
svm_model = joblib.load("/content/svm_filter_selector.pkl")

# Initialize video capture
video_path = "/content/drive/MyDrive/traffic.mp4"
cap = cv2.VideoCapture(video_path)

# Define video output
output_video_path = "/content/drive/MyDrive/n_tracking.mp4"
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Vehicle class mapping
class_names = {0: "Car", 1: "Truck", 2: "Bus", 3: "Train"}

# Tracking variables
trackers = {}  # Stores Kalman filters for each tracked vehicle
track_history = {}  # Stores past positions for trajectory visualization
next_id = 1  # Unique tracking ID counter

# 🔹 Function to Initialize Kalman Filter
def create_kf():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1, 1, 0, 0],  # State transition matrix
                     [0, 1, 0, 0],
                     [0, 0, 1, 1],
                     [0, 0, 0, 1]])
    kf.H = np.array([[1, 0, 0, 0],  # Measurement function
                     [0, 0, 1, 0]])
    kf.P *= 1000  # High uncertainty
    kf.R *= 10    # Measurement noise
    kf.Q *= 0.1   # Process noise
    return kf

# 🔹 Function to Select the Best Kalman Filter Based on Motion Features
def select_filter(velocity, acceleration, direction_change):
    features = np.array([[velocity, acceleration, direction_change]])
    filter_type = svm_model.predict(features)[0]

    if filter_type == 0:
        return create_kf()  # Kalman Filter (KF)
    elif filter_type == 1:
        return create_kf()  # Extended Kalman Filter (Placeholder)
    else:
        return create_kf()  # Unscented Kalman Filter (Placeholder)

# 🔹 Function to Apply Error Feedback Mechanism
def error_feedback(kf, error):
    if error > 20:
        kf.Q *= 1.5  # Increase process noise
    elif error < 5:
        kf.Q *= 0.8  # Decrease process noise
    return kf

# 🔹 Function to Calculate Intersection over Union (IoU)
def iou(box1, box2):
    x1, y1, x2, y2 = box1
    x3, y3, x4, y4 = box2
    xi1, yi1, xi2, yi2 = max(x1, x3), max(y1, y3), min(x2, x4), min(y2, y4)
    inter_area = max(0, xi2 - xi1) * max(0, yi2 - yi1)
    box1_area = (x2 - x1) * (y2 - y1)
    box2_area = (x4 - x3) * (y4 - y3)
    union_area = box1_area + box2_area - inter_area
    return inter_area / union_area if union_area > 0 else 0

# 🔹 Process Video Frame by Frame
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Run YOLOv8 detection
    results = model(frame)

    detections = []
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
            conf = box.conf[0].cpu().numpy()
            cls = int(box.cls[0].cpu().numpy())

            # Only track vehicles (Car=0, Truck=1, Bus=2, Train=3)
            if cls in class_names:
                detections.append([x1, y1, x2, y2, conf, cls])

    # Convert to numpy array
    detections = np.array(detections)

    # Tracking Logic
    new_trackers = {}
    assigned_ids = set()

    for detection in detections:
        x1, y1, x2, y2, conf, cls = detection
        center_x = (x1 + x2) / 2
        center_y = (y1 + y2) / 2

        # Compute motion features
        velocity = np.random.uniform(0, 10)  # Placeholder velocity computation
        acceleration = np.random.uniform(0, 5)  # Placeholder acceleration
        direction_change = np.random.uniform(0, 30)  # Placeholder

        matched_id = None
        for track_id, kf in trackers.items():
            predicted_x, predicted_y = int(kf.x[0, 0]), int(kf.x[2, 0])
            dist = np.linalg.norm([center_x - predicted_x, center_y - predicted_y])

            if dist < 50 and track_id not in assigned_ids:
                matched_id = track_id
                assigned_ids.add(matched_id)
                break

        if matched_id is None:
            matched_id = next_id
            next_id += 1
            kf = select_filter(velocity, acceleration, direction_change)
            kf.x[:2] = np.array([[center_x], [center_y]])
        else:
            kf = trackers[matched_id]

        # Predict and update Kalman filter
        state = kf.predict()
        if state is None:
            state = kf.x
        kf.update(np.array([center_x, center_y]))

        # Apply error feedback
        tracking_error = np.linalg.norm([center_x - state[0], center_y - state[1]])
        kf = error_feedback(kf, tracking_error)

        new_trackers[matched_id] = kf

        # Store tracking history for trajectory visualization
        if matched_id not in track_history:
            track_history[matched_id] = []
        track_history[matched_id].append((int(center_x), int(center_y)))

        if len(track_history[matched_id]) > 30:
            track_history[matched_id].pop(0)

        # Draw trajectory line
        for i in range(1, len(track_history[matched_id])):
            cv2.line(frame, track_history[matched_id][i - 1], track_history[matched_id][i], (255, 0, 255), 2)

        # Draw bounding box and label
        label = f"{matched_id}:{class_names[cls]}"
        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
        cv2.putText(frame, label, (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

    trackers = new_trackers  # Update tracker dictionary
    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"✅ Adaptive Kalman Tracking Complete! Saved video: {output_video_path}")


In [ ]:
!pip install torchmetrics

In [ ]:
# Step 1: Import necessary libraries
import torch
from torchmetrics.detection import MeanAveragePrecision

# Step 2: Define Ground Truth (from KITTI labels)
ground_truths = [
    {
        "boxes": torch.tensor([[50, 50, 200, 200], [300, 100, 400, 250]], dtype=torch.float32),
        "labels": torch.tensor([0, 1], dtype=torch.int64),  # 0: Car, 1: Truck
    }
]

# Step 3: Define Predictions (from YOLOv8 model output)
predictions = [
    {
        "boxes": torch.tensor([[55, 55, 195, 195], [310, 110, 390, 240]], dtype=torch.float32),
        "scores": torch.tensor([0.9, 0.85], dtype=torch.float32),  # Confidence scores
        "labels": torch.tensor([0, 1], dtype=torch.int64),  # Class IDs
    }
]

# Step 4: Compute mAP
metric = MeanAveragePrecision(iou_type="bbox")  # Initialize metric
metric.update(predictions, ground_truths)  # Update with data
results = metric.compute()  # Compute mAP

# Step 5: Print mAP Results
print("mAP:", results["map"].item())  # Mean Average Precision
print("mAP@50:", results["map_50"].item())  # IoU=0.50
print("mAP@75:", results["map_75"].item())  # IoU=0.75


In [ ]:
def calculate_iou(box1, box2):
    """
    Compute Intersection over Union (IoU) between two bounding boxes.

    Parameters:
        box1 (list): [x1, y1, x2, y2] for ground truth.
        box2 (list): [x1, y1, x2, y2] for prediction.

    Returns:
        iou (float): Intersection over Union score.
    """
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])

    intersection = max(0, x2 - x1) * max(0, y2 - y1)  # Area of overlap
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])  # GT area
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])  # Pred area
    union = box1_area + box2_area - intersection  # Area of union

    iou = intersection / union if union != 0 else 0  # Avoid division by zero
    return iou


def calculate_mota(gt_tracks, pred_tracks):
    """
    Compute MOTA (Multiple Object Tracking Accuracy).

    Parameters:
        gt_tracks (dict): Ground truth tracks {frame_id: {object_id: [x1, y1, x2, y2]}}.
        pred_tracks (dict): Predicted tracks {frame_id: {object_id: [x1, y1, x2, y2]}}.

    Returns:
        mota (float): MOTA score.
    """
    FP = 0  # False Positives
    FN = 0  # False Negatives
    IDS = 0 # ID Switches
    GT = 0  # Total Ground Truth Objects
    prev_matches = {}  # Store ID assignments across frames

    for frame_id in gt_tracks.keys():
        gt_objects = gt_tracks.get(frame_id, {})  # GT for this frame
        pred_objects = pred_tracks.get(frame_id, {})  # Predictions for this frame
        GT += len(gt_objects)  # Count total GT objects

        matched_gt = set()
        matched_pred = set()

        for gt_id, gt_bbox in gt_objects.items():
            for pred_id, pred_bbox in pred_objects.items():
                iou = calculate_iou(gt_bbox, pred_bbox)
                if iou > 0.5:  # Consider IoU > 0.5 as a valid match
                    matched_gt.add(gt_id)
                    matched_pred.add(pred_id)

                    # Check for ID switches
                    if gt_id in prev_matches and prev_matches[gt_id] != pred_id:
                        IDS += 1  # ID switched

                    prev_matches[gt_id] = pred_id

        # False Negatives: GT objects not matched
        FN += len(gt_objects) - len(matched_gt)

        # False Positives: Predictions not matched
        FP += len(pred_objects) - len(matched_pred)

    # Compute MOTA
    mota = 1 - ((FP + FN + IDS) / max(GT, 1))  # Avoid division by zero
    return mota


# Example Usage
gt_tracks = {
    1: {1: [50, 50, 100, 100], 2: [150, 150, 200, 200]},  # Frame 1 GT
    2: {1: [55, 55, 105, 105], 2: [155, 155, 205, 205]}   # Frame 2 GT
}

pred_tracks = {
    1: {1: [52, 52, 98, 98], 2: [148, 148, 202, 202]},  # Frame 1 Prediction
    2: {1: [56, 56, 106, 106], 2: [154, 154, 204, 204]}  # Frame 2 Prediction
}

mota = calculate_mota(gt_tracks, pred_tracks)
print(f"MOTA: {mota:.4f}")


In [ ]:
import torch
from torchmetrics.detection import MeanAveragePrecision

# Define ground truth and predicted boxes (Example format)
ground_truths = [
    {"boxes": torch.tensor([[50, 50, 200, 200]]), "labels": torch.tensor([0])}
]
predictions = [
    {"boxes": torch.tensor([[55, 55, 195, 195]]), "scores": torch.tensor([0.9]), "labels": torch.tensor([0])}
]

# Compute mAP
metric = MeanAveragePrecision()
metric.update(predictions, ground_truths)
results = metric.compute()

print("mAP: ", results["map"].item())  # Convert to float


In [ ]:
from torchmetrics.detection import MeanAveragePrecision

metric = MeanAveragePrecision()

ground_truths = [
    {"boxes": torch.tensor([[50, 50, 100, 100], [150, 150, 200, 200]], dtype=torch.float32),
     "labels": torch.tensor([1, 2], dtype=torch.int64)}
]

predictions = [
    {"boxes": torch.tensor([[52, 52, 98, 98], [148, 148, 202, 202]], dtype=torch.float32),
     "scores": torch.tensor([0.9, 0.8], dtype=torch.float32),
     "labels": torch.tensor([1, 2], dtype=torch.int64)}
]

metric.update(predictions, ground_truths)
result = metric.compute()

print(f"mAP: {result['map']}")


In [ ]:
import time
import cv2

video_path = "/content/drive/MyDrive/n_tracking.mp4"  # Change to 0 for webcam
cap = cv2.VideoCapture(video_path)

frame_count = 0
start_time = time.time()

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break  # Stop if video ends

    # Run YOLO detection/tracking on each frame
    results = model.track(frame, persist=True)  # Example: YOLOv8 tracking

    frame_count += 1

end_time = time.time()
total_time = end_time - start_time
fps = frame_count / total_time

print(f"Average FPS: {fps:.2f}")

cap.release()
cv2.destroyAllWindows()


In [ ]:
import numpy as np
from filterpy.kalman import KalmanFilter

def create_kf():
    kf = KalmanFilter(dim_x=4, dim_z=2)
    kf.F = np.array([[1, 0, 1, 0],  # State transition matrix
                     [0, 1, 0, 1],
                     [0, 0, 1, 0],
                     [0, 0, 0, 1]])
    kf.H = np.array([[1, 0, 0, 0],  # Observation matrix
                     [0, 1, 0, 0]])
    kf.R *= 0.05  # Measurement noise
    kf.P *= 1.0  # Initial state covariance
    kf.Q *= 0.1  # Process noise
    return kf


In [ ]:
from sklearn.svm import SVC

# Train an SVM model for motion classification
X_train = np.random.rand(100, 3)  # Dummy feature data [velocity, acceleration, curvature]
y_train = np.random.choice([0, 1, 2], 100)  # Labels (0: KF, 1: EKF, 2: UKF)

svm_model = SVC(kernel='rbf')
svm_model.fit(X_train, y_train)

# Function to select filter dynamically
def select_filter(features):
    label = svm_model.predict([features])[0]
    if label == 0:
        return create_kf()
    elif label == 1:
        return create_ekf()  # Define create_ekf similar to create_kf
    else:
        return create_ukf()  # Define create_ukf similar to create_kf


In [ ]:
def error_feedback(kf, error):
    kf.Q *= (1 + 0.1 * abs(error))  # Adjust process noise
    kf.R *= (1 + 0.1 * abs(error))  # Adjust measurement noise
    return kf


In [ ]:
import cv2

video_path = "/content/drive/MyDrive/KIITI YOLO DATASET/testing/videos/sample.mp4"
cap = cv2.VideoCapture(video_path)

kf = create_kf()  # Initialize Kalman filter

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLOv8 detection
    results = model(frame)

    for result in results:  # Iterate over the list of Results objects
        detections = result.boxes  # Access detections

        for detection in detections:
            x, y, w, h = detection.xywh[0]  # Get bounding box (center_x, center_y, width, height)
            conf = detection.conf[0]  # Confidence score
            cls = detection.cls[0]  # Class label

            # Select filter dynamically based on motion
            features = [x, y, w]  # Example features
            kf = select_filter(features)

            # Apply Kalman filtering
            state = kf.predict()
            kf.update(np.array([x, y]))

            # Apply error feedback mechanism
            error = np.linalg.norm([x - state[0], y - state[1]])
            kf = error_feedback(kf, error)

            # Draw tracking results
            cv2.rectangle(frame, (int(x - w/2), int(y - h/2)), (int(x + w/2), int(y + h/2)), (0, 255, 0), 2)

    from google.colab.patches import cv2_imshow

cap.release()  # Release video capture

# Show the last frame in Colab
cv2_imshow(frame)



In [ ]:
from ultralytics import YOLO

# Load your pretrained YOLOv8 model
model = YOLO("/content/yolov8m.pt")  # Replace with your model path

# Validate model on labeled dataset
metrics = model.val(data="/content/drive/MyDrive/KIITI YOLO DATASET/data.yaml")

# Extract and print mAP scores
print(f"mAP@0.5: {metrics.box.map50:.4f}")  # IoU=0.5
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")  # IoU=0.5:0.95
